##Validando a SparkSession

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

##Conectando Azure ADLS Gen2 no Databricks

###Mostrando os pontos de montagem no cluster Databricks

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/mnt/datalake6d3129a9a4390b0a/gold,wasbs://gold@datalake6d3129a9a4390b0a.blob.core.windows.net,
/mnt/datalakeedda2e0be8d3ec5f/bronze,wasbs://bronze@datalakeedda2e0be8d3ec5f.blob.core.windows.net,
/mnt/datalake246d728b13561613/lading-zone,wasbs://lading-zone@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalake4b6c87c48101c278/bronze,wasbs://bronze@datalake4b6c87c48101c278.blob.core.windows.net,
/mnt/datalake4b6c87c48101c278/gold,wasbs://gold@datalake4b6c87c48101c278.blob.core.windows.net,
/databricks-datasets,databricks-datasets,
/mnt/datalake246d728b13561613/bronze,wasbs://bronze@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalakee66472c14f9678fd/landing-zone,wasbs://landing-zone@datalakee66472c14f9678fd.blob.core.windows.net,
/databricks/mlflow-tracking,databricks/mlflow-tracking,sse-s3
/mnt/datalake4b6c87c48101c278/silver,wasbs://silver@datalake4b6c87c48101c278.blob.core.windows.net,


###Desmontando os pontos de montagem não utilizados

In [0]:
#dbutils.fs.unmount('/mnt/datalake6f2d8d16eba38233/bronze')
#dbutils.fs.unmount('/mnt/datalakefc6082cb60bef06c/bronze')

### Definindo uma função para montar um ADLS com um ponto de montagem com ADLS SAS 

In [0]:
storageAccountName = "datalake4b6c87c48101c278"
storageAccountAccessKey = ""
sasToken = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-07T12:17:21Z&st=2025-06-07T04:17:21Z&spr=https&sig=BfwzYwM%2B5YR%2FBkSSyRv0Nn%2F3riHK9Wx4P%2FQoguM%2BA%2FM%3D"

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)

###Montando todos os containers

In [0]:
mount_adls('lading-zone')
mount_adls('bronze')
mount_adls('silver')
mount_adls('gold')

OK!
Falha An error occurred while calling o1919.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake4b6c87c48101c278/bronze; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake4b6c87c48101c278/bronze
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:135)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:1053)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.$anonfun$mount$1(DBUtilsCore.scala:1079)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:560)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:657)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLoggin

###Mostrando os pontos de montagem no cluster Databricks

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/mnt/datalake6d3129a9a4390b0a/gold,wasbs://gold@datalake6d3129a9a4390b0a.blob.core.windows.net,
/mnt/datalakeedda2e0be8d3ec5f/bronze,wasbs://bronze@datalakeedda2e0be8d3ec5f.blob.core.windows.net,
/mnt/datalake246d728b13561613/lading-zone,wasbs://lading-zone@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalake4b6c87c48101c278/bronze,wasbs://bronze@datalake4b6c87c48101c278.blob.core.windows.net,
/mnt/datalake4b6c87c48101c278/gold,wasbs://gold@datalake4b6c87c48101c278.blob.core.windows.net,
/databricks-datasets,databricks-datasets,
/mnt/datalake246d728b13561613/bronze,wasbs://bronze@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalakee66472c14f9678fd/landing-zone,wasbs://landing-zone@datalakee66472c14f9678fd.blob.core.windows.net,
/databricks/mlflow-tracking,databricks/mlflow-tracking,sse-s3
/mnt/datalake4b6c87c48101c278/silver,wasbs://silver@datalake4b6c87c48101c278.blob.core.windows.net,


### Mostrando todos os arquivos da camada bronze

In [0]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze"))

path,name,size,modificationTime
dbfs:/mnt/datalake4b6c87c48101c278/bronze/customers/,customers/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/geolocation/,geolocation/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/leads_closed/,leads_closed/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/leads_qualified/,leads_qualified/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/order_items/,order_items/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/order_payments/,order_payments/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/order_reviews/,order_reviews/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/orders/,orders/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/product_category_name_translation/,product_category_name_translation/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/products/,products/,0,0


###Gerando um dataframe dos delta lake no container bronze do Azure Data Lake Storage

In [0]:
df_customers   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/customers")
df_geolocation     = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/geolocation")
df_leads_closed   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/leads_closed")
df_leads_qualified  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/leads_qualified")
df_order_items    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/order_items")
df_order_payments     = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/order_payments")
df_order_reviews    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/order_reviews")
df_orders = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/orders")
df_product_category_name_translation    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/product_category_name_translation")
df_products  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/products")
df_sellers  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/sellers")

### Adicionando metadados de data e hora de processamento e nome do arquivo de origem

In [0]:
from pyspark.sql.functions import current_timestamp, lit

df_customers   = df_customers.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("customers"))
df_geolocation     = df_geolocation.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("geolocation"))
df_leads_closed   = df_leads_closed.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("leads_closed"))
df_leads_qualified  = df_leads_qualified.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("leads_qualified"))
df_order_items    = df_order_items.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("order_items"))
df_order_payments     = df_order_payments.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("order_payments"))
df_order_reviews    = df_order_reviews.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("order_reviews"))
df_orders = df_orders.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("orders"))
df_product_category_name_translation    = df_product_category_name_translation.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("product_category_name_translation"))
df_products  = df_products.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("products"))
df_sellers  = df_sellers.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("sellers"))


In [0]:
def imprimir_colunas_maiusculas(dataframes):
    for nome_df, df in dataframes:
        print(f"\nColunas do DataFrame '{nome_df}' em maiúsculas:")
        colunas_maiusculas = [coluna.upper() for coluna in df.columns]
        for coluna in colunas_maiusculas:
            print(coluna)

# Lista com tuplas (nome do DataFrame, DataFrame em si)
dataframes = [
    ("df_customers", df_customers),
    ("df_geolocation", df_geolocation),
    ("df_leads_closed", df_leads_closed),
    ("df_leads_qualified", df_leads_qualified),
    ("df_order_items", df_order_items),
    ("df_order_payments", df_order_payments),
    ("df_order_reviews", df_order_reviews),
    ("df_orders", df_orders),
    ("df_product_category_name_translation", df_product_category_name_translation),
    ("df_products", df_products),
    ("df_sellers", df_sellers)
]

# Chamada da função
imprimir_colunas_maiusculas(dataframes)


Colunas do DataFrame 'df_customers' em maiúsculas:
CUSTOMER_ID
CUSTOMER_UNIQUE_ID
CUSTOMER_ZIP_CODE_PREFIX
CUSTOMER_CITY
CUSTOMER_STATE
DATA_HORA_BRONZE
NOME_ARQUIVO
DATA_HORA_SILVER

Colunas do DataFrame 'df_geolocation' em maiúsculas:
GEOLOCATION_ZIP_CODE_PREFIX
GEOLOCATION_LAT
GEOLOCATION_LNG
GEOLOCATION_CITY
GEOLOCATION_STATE
DATA_HORA_BRONZE
NOME_ARQUIVO
DATA_HORA_SILVER

Colunas do DataFrame 'df_leads_closed' em maiúsculas:
MQL_ID
SELLER_ID
SDR_ID
SR_ID
WON_DATE
BUSINESS_SEGMENT
LEAD_TYPE
LEAD_BEHAVIOUR_PROFILE
HAS_COMPANY
HAS_GTIN
AVERAGE_STOCK
BUSINESS_TYPE
DECLARED_PRODUCT_CATALOG_SIZE
DECLARED_MONTHLY_REVENUE
DATA_HORA_BRONZE
NOME_ARQUIVO
DATA_HORA_SILVER

Colunas do DataFrame 'df_leads_qualified' em maiúsculas:
MQL_ID
FIRST_CONTACT_DATE
LANDING_PAGE_ID
ORIGIN
DATA_HORA_BRONZE
NOME_ARQUIVO
DATA_HORA_SILVER

Colunas do DataFrame 'df_order_items' em maiúsculas:
ORDER_ID
ORDER_ITEM_ID
PRODUCT_ID
SELLER_ID
SHIPPING_LIMIT_DATE
PRICE
FREIGHT_VALUE
DATA_HORA_BRONZE
NOME_ARQUIVO
DAT

###Mudando as colunas para maiscula e ajustanto os nomes das colunas de acordo com o dicionario de dados

df_apolice = (
	df_apolice
	.withColumnRenamed("cd_apolice"         , "CODIGO_APOLICE")
	.withColumnRenamed("cd_cliente"         , "CODIGO_CLIENTE")
	.withColumnRenamed("dt_inicio_vigencia" , "DATA_INICIO_VIGENCIA")
	.withColumnRenamed("dt_fim_vigencia"    , "DATA_FIM_VIGENCIA")
	.withColumnRenamed("vl_cobertura"       , "VALOR_COBERTURA")
	.withColumnRenamed("vl_franquia"        , "VALOR_FRANQUIA")
	.withColumnRenamed("placa"              , "PLACA")
	.drop("data_hora_bronze")
	.withColumnRenamed("nome_arquivo"       , "NOME_ARQUIVO_BRONZE")
	.withColumnRenamed("data_hora_silver"   , "DATA_HORA_SILVER")
)

###Visualiza o dataframe atualizado para silver

df_apolice.display()

###Salvando os dataframes em delta lake (formato de arquivo) no data lake (repositorio cloud)

df_apolice.write.format('delta').save(f"/mnt/{storageAccountName}/silver/apolice")

###Verificando os dados gravados em delta na camada silver

display(dbutils.fs.ls(f"/mnt/{storageAccountName}/silver/"))

### Lendo um exemplo de um delta lake para validar a existencia dos dados e das colunas do metadados

spark.read.format('delta').load(f'/mnt/{storageAccountName}/silver/apolice').limit(10).display()

#### BONUS - Efetuando a mesma atividade da silver (listada acima) de forma massiva em todos os arquivos delta

### Fazer as transformacoes de forma massiva

In [0]:
#Fucao para renomear as colunas de cada delta table para maiuscula e ajusta o nome das colunas de acordo com o dicionario de dados
def renomear_colunas(diretorio):

    # Carregue o DataFrame a partir do delta
    df = spark.read.format('delta').load(diretorio)

    tabela = diretorio.split('/')[-2]

    # Renomeie todas as colunas para maiúsculas e faça a substituição do padrão "cd_" para "codigo_"
    for coluna in df.columns:
        novo_nome = coluna.upper()
        novo_nome = novo_nome.replace("_ID", "_CODIGO")
        novo_nome = novo_nome.replace("_QTY", "_QUANTIDADE")
        novo_nome = novo_nome.replace("_LAT", "_LATITUDE")
        novo_nome = novo_nome.replace("_LNG", "_LONGITUDE")
        novo_nome = novo_nome.replace("_CITY", "_CIDADE")
        novo_nome = novo_nome.replace("_STATE", "_ESTADO")
        novo_nome = novo_nome.replace("_DATE", "_DATA")
        novo_nome = novo_nome.replace("_SEGMENT", "_SEGMENTO")
        novo_nome = novo_nome.replace("_TYPE", "_TIPO")
        novo_nome = novo_nome.replace("_PROFILE", "_PERFIL")
        novo_nome = novo_nome.replace("_COMPANY", "_EMPRESA")
        novo_nome = novo_nome.replace("_SIZE", "_TAMANHO")
        novo_nome = novo_nome.replace("_VALUE", "_VALOR")
        novo_nome = novo_nome.replace("_SEQUENTIAL", "_SEQUENCIA")
        novo_nome = novo_nome.replace("_SCORE", "_PONTUACAO")
        novo_nome = novo_nome.replace("_TITLE", "_TITULO")
        novo_nome = novo_nome.replace("_MESSAGE", "_MENSAGEM")
        novo_nome = novo_nome.replace("_INSTALLMENTS", "_PARCELAS")
        novo_nome = novo_nome.replace("_NAME", "_NOME")
        novo_nome = novo_nome.replace("_LENGHT", "_COMPRIMENTO")
        novo_nome = novo_nome.replace("_G", "_GRAMAS")
        novo_nome = novo_nome.replace("_CM", "_CENTIMETROS")
        novo_nome = novo_nome.replace("_PREFIX", "_PREFIXO")

        df = df.withColumnRenamed(coluna, novo_nome)
        df = df.drop("DATA_HORA_BRONZE")
        df = df.drop("NOME_ARQUIVO")
        df = df.withColumn("NOME_ARQUIVO_BRONZE", lit(tabela))
        df = df.withColumn("DATA_ARQUIVO_SILVER", current_timestamp())

    # Salve o DataFrame modificado de volta no mesmo local
    #df.display()
    df.write.format('delta').save(f"/mnt/{storageAccountName}/silver/{tabela}")

#Funcao que chama a funcao renomear colunas para todos os arquivos contidos no container
def renomear_arquivos_delta(diretorio):

    # Lista para armazenar os nomes dos arquivos delta
    nomes_arquivos_delta = []

    # Lista os arquivos no diretório
    arquivos = dbutils.fs.ls(diretorio)
    
    # Itera sobre os arquivos e armazena os nomes dos arquivos delta
    for arquivo in arquivos:
        nome_arquivo = arquivo.path
        # Remover as aspas e a barra (/) no final do nome do arquivo
        #nome_arquivo = nome_arquivo.rstrip('/')
        #nomes_arquivos_delta.append(nome_arquivo)
        renomear_colunas(nome_arquivo)
        #print(nome_arquivo)

    return nomes_arquivos_delta

In [0]:
# Executa a funcao para atualizar todos os dataframes
diretorio = f'/mnt/{storageAccountName}/bronze'

renomear_arquivos_delta(diretorio)

Out[38]: []

In [0]:
spark.read.format('delta').load(f'/mnt/{storageAccountName}/silver/customers').limit(10).display()

CUSTOMER_CODIGO,CUSTOMER_UNIQUE_CODIGO,CUSTOMER_ZIP_CODE_PREFIXO,CUSTOMER_CIDADE,CUSTOMER_ESTADO,NOME_ARQUIVO_BRONZE,DATA_ARQUIVO_SILVER
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,customers,2025-06-07T04:30:16.007+0000
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,customers,2025-06-07T04:30:16.007+0000
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,customers,2025-06-07T04:30:16.007+0000
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,customers,2025-06-07T04:30:16.007+0000
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,customers,2025-06-07T04:30:16.007+0000
879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC,customers,2025-06-07T04:30:16.007+0000
fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,sao paulo,SP,customers,2025-06-07T04:30:16.007+0000
5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG,customers,2025-06-07T04:30:16.007+0000
5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR,customers,2025-06-07T04:30:16.007+0000
4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG,customers,2025-06-07T04:30:16.007+0000


In [0]:
#Funcao auxiliar
import os

# Use a função para renomear as colunas para maiúsculo em um diretório específico
diretorio = f'/mnt/{storageAccountName}/bronze/'

arquivos = dbutils.fs.ls(diretorio)

# Imprime os nomes dos arquivos
print("Arquivos no diretório:")
for arquivo in arquivos:
    print(arquivo.name.rstrip('/'), arquivo.name.rstrip('/').upper())

Arquivos no diretório:
customers CUSTOMERS
geolocation GEOLOCATION
leads_closed LEADS_CLOSED
leads_qualified LEADS_QUALIFIED
order_items ORDER_ITEMS
order_payments ORDER_PAYMENTS
order_reviews ORDER_REVIEWS
orders ORDERS
product_category_name_translation PRODUCT_CATEGORY_NAME_TRANSLATION
products PRODUCTS
sellers SELLERS
